# 가위바위보 이미지 분류하기

1. 데이터를 준비하자

In [1]:
from PIL import Image
import glob

print("PIL 라이브러리 import 완료!")

PIL 라이브러리 import 완료!


In [2]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

	target_size=(28,28) #이미지 28*28 사이즈로 통일
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")

import os
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/scissor1"
resize_images(image_dir_path)

print("가위 이미지 resize 완료!")

102  images to be resized.
102  images resized.
가위 이미지 resize 완료!


In [3]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")


	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/rock1"
resize_images(image_dir_path)

print("주먹 이미지 resize 완료!")

105  images to be resized.
105  images resized.
주먹 이미지 resize 완료!


In [4]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")


	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/paper1"
resize_images(image_dir_path)

print("보 이미지 resize 완료!")

107  images to be resized.
107  images resized.
보 이미지 resize 완료!


In [14]:
import numpy as np

def load_data(img_path, number_of_data=314):  

    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor1/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1
  
    for file in glob.iglob(img_path+'/rock1/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
        
  
    for file in glob.iglob(img_path+'/paper1/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper"
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 314 입니다.
x_train shape: (314, 28, 28, 3)
y_train shape: (314,)


2. 딥러닝 네트워크 설계하기

In [27]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

n_channel_1=8
n_channel_2=16
n_dense=64
n_train_epoch=10

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

print('Model에 추가된 Layer 개수: ', len(model.layers))

model.summary()

Model에 추가된 Layer 개수:  7
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 26, 26, 8)         224       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 13, 13, 8)         0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 11, 11, 16)        1168      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                25664     
_________________________________________________________________
dense_11 (Dense)             (

3. 딥러닝 네트워크 학습시키기

In [28]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=n_train_epoch)

Epoch 1/10
10/10 [==============================] - 0s 3ms/step - loss: 9.3052 - accuracy: 0.4713
Epoch 2/10
10/10 [==============================] - 0s 3ms/step - loss: 2.0898 - accuracy: 0.6592
Epoch 3/10
10/10 [==============================] - 0s 3ms/step - loss: 1.0899 - accuracy: 0.7261
Epoch 4/10
10/10 [==============================] - 0s 3ms/step - loss: 0.7406 - accuracy: 0.7866
Epoch 5/10
10/10 [==============================] - 0s 3ms/step - loss: 0.4798 - accuracy: 0.8503
Epoch 6/10
10/10 [==============================] - 0s 3ms/step - loss: 0.2382 - accuracy: 0.9172
Epoch 7/10
10/10 [==============================] - 0s 3ms/step - loss: 0.2169 - accuracy: 0.9204
Epoch 8/10
10/10 [==============================] - 0s 3ms/step - loss: 0.1187 - accuracy: 0.9554
Epoch 9/10
10/10 [==============================] - 0s 3ms/step - loss: 0.0808 - accuracy: 0.9682
Epoch 10/10
10/10 [==============================] - 0s 3ms/step - loss: 0.0550 - accuracy: 0.9777


4. 테스트해보기

In [29]:
# x_test, y_test
import numpy as np

def load_data(img_path, number_of_data=313):  
  
    img_size=28
    color=3
  
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/s1/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/r1/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
 
    for file in glob.iglob(img_path+'/p1/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
        
    print("학습데이터(x_test)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test"
(x_test, y_test)=load_data(image_dir_path)
x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

학습데이터(x_test)의 이미지 개수는 313 입니다.
x_test shape: (313, 28, 28, 3)
y_test shape: (313,)


In [30]:
test_loss, test_accuracy = model.evaluate(x_test,y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

10/10 - 0s - loss: 2.1188 - accuracy: 0.6518
test_loss: 2.118803024291992 
test_accuracy: 0.6517571806907654


# 5. 회고

# 첫번째,
정확도가 높게 나오려면 많은 데이터 학습이 중요하다고 생각했다.
그래서 학습용 데이터 7000개(4명) 테스트 데이터 300개로 분별하였다.
>>결과 : 정확도 0.3~0.4

고칠점
1. 너무 많은 학습용 데이터
2. 적당한 파라미터 값 ex) n_channel_1=8, n_channel_2=16, ,n_dense=16
3. 에폭값 줄이기 n_train_epoch=8

# 두번째,
학습용 데이터 사진 중 판별 불가한 (포커싱이 잡히지 않은 사진, 너무 작게 나온 사진 등) 사진들을 삭제하고 데이터 수를 줄였다.

train set: 7000->3000 / test set:300

>>결과 : 정확도  0.3~0.5

# 세번째,
계속해도 높지 않은 정확도에 의문을 가져
순수하게 두사람의 데이터로만 하면 높게나오지 않을까해서 시도해보았다.

train set: 300 / test set: 300

>>결과 : 정확도 0.4~

# 네번째,
이번에는 학습용 데이터와 테스트 데이터를 랜덤으로 나누어 보았다.

train set: 150+150 / test set: 150+150

>>결과 : 정확도 0.5~0.65

# 총정리

파라미터값을 아무리 조정해도 높은 값을 기대해보기엔 어려웠다.
그래서 내린 결론은 데이터를 조정해보는 것이었다.

-학습 데이터 중 가위바위보 이미지에 벗어났다고 생각한 것들을 추리는 방법

-데이터 수를 늘리거나 줄이는 방법

-학습용/테스트용을 랜덤으로 나누는 방법


결국 높은 결과는 나오지 못했지만 정확도를 높이기 위해 시도한 여러 과정들이 의미있었다고 생각한다.